In [1]:
from torch.utils.data import DataLoader
import os
os.chdir('../')
os.environ["TRANSFORMERS_CACHE"]="/common/users/zl502/huggingface/cache/"

import torch

# from peft import PeftModel, PeftModelForCausalLM, prepare_model_for_int8_training, LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoFeatureExtractor

from general_dataset import GeneralDataset
from agi_utils import *
from tqdm import tqdm
from undecorated import undecorated
from types import MethodType


import numpy as np
from IPython.utils import io
import random
from evaluate import load
from torchvision import transforms
from torchmetrics.multimodal import CLIPScore
from combine_model_seq import SeqCombine

In [2]:
"""
assign openagi data path 
"""
# data_path = "YOUR_DATA_PATH"
data_path = "/common/users/zl502/openagi_data/"

task_discriptions = txt_loader("./task_description.txt")
# test_task_idx = [0,21,61,105,110,120,10,35,62,107,115]
test_task_idx = [2,3,10,15,20,35,45,55,65,70,70,90,106,107]
# test_task_idx = list(range(183))
test_dataloaders = []
for i in tqdm(test_task_idx):
    dataset = GeneralDataset(i, data_path)
    dataloader = DataLoader(dataset, batch_size=20)
    test_dataloaders.append(dataloader)
    
test_tasks = [task_discriptions[i].strip() for i in test_task_idx]

100%|███████████████████████████████████████████| 14/14 [00:05<00:00,  2.50it/s]


In [3]:
# base_model = "eachadea/vicuna-7b-1.1"
# base_model = "NousResearch/Nous-Hermes-Llama2-13b"
# base_model = "chainyo/alpaca-lora-7b"
base_model = "meta-llama/Llama-2-13b-chat-hf"
# base_model = "meta-llama/Llama-2-70b-chat-hf"
# base_model = "TheBloke/Llama-2-13B-chat-GGML"
# base_model = "chainyo/alpaca-lora-7b"
load_8bit = True

hf_token = "YOUR TOKEN"

max_memory_mapping = {
    0: "0GB",
    1: "0GB",
    2: "0GB",
    3: "0GB",
    4: "48GB",
    5: "48GB",
    6: "48GB",
    7: "48GB",
}

# max_memory_mapping = {
#     0: "0GB",
#     1: "0GB",
#     2: "24GB",
#     3: "24GB",
# }

tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    cache_dir="/common/users/zl502/huggingface/cache",
    use_auth_token=hf_token
    # padding_side='left'
)
# tokenizer.add_special_tokens({'pad_token': '<pad>'})
tokenizer.pad_token_id = 0

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    cache_dir="/common/users/zl502/huggingface/cache",
    device_map="auto",
    max_memory=max_memory_mapping,
    use_auth_token=hf_token
)


model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# output["scores"]
import openai
# prompt_length = len(input_text)
openai.api_key = "YOUR TOKEN"

def generate_module_list_with_gpt(generated_module_seq):
    todo_prompt = "You are a key phrase extractor who is able to extract potential module names from the given context. You have already known all the module names in the full module list. The full module list is: [Image Classification, Colorization, Object Detection, Image Deblurring, Image Denoising, Image Super Resolution, Image Captioning, Text to Image Generation, Visual Question Answering, Sentiment Analysis, Question Answering, Text Summarization, Machine Translation]. Given the following context: '{}'. Please extract a module sequence from this context and remove module names which do not exist in the full module list from this sequence. Output the module sequence after filtering as the format of 'module: module1, module: module2, module: module3, etc...'. "
    prompt = todo_prompt.format(generated_module_seq)

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )

    content = completion.choices[0].message["content"]
    
    # print(content)
    
    content = content.split("module: ")[1:]
    
    result = ""
    for c in content:
        result += c
    
    # result = result[:-1] if len(result) > 0 else result
    
    return result

# generated_module_list = generate_module_list_with_gpt(response[prompt_length:])
# print(generated_module_list)

In [7]:
"""
Loading Evaluation Metrics
"""

clip_score = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")


# Load a pre-trained Vision Transformer model and its feature extractor
vit_ckpt = "nateraw/vit-base-beans"
vit = AutoModel.from_pretrained(vit_ckpt)
vit.eval()
vit_extractor = AutoFeatureExtractor.from_pretrained(vit_ckpt)

f = transforms.ToPILImage()
bertscore = load("bertscore")

# device_list = ["cuda:1","cuda:2","caugment_prompt:3","cuda:4","cuda:5","cuda:7","cpu"]
device_list = ["cuda:0", "cpu"]
seqCombination = SeqCombine(device_list)

Some weights of the model checkpoint at nateraw/vit-base-beans were not used when initializing ViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at nateraw/vit-base-beans and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fetching 20 files:   0%|          | 0/20 [00:00<?, ?it/s]

The config attributes {'scaling_factor': 0.18215} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.


In [8]:
from sentence_transformers import SentenceTransformer, util
sentence_model = SentenceTransformer('all-MiniLM-L6-v2', device="cpu")


rewards = []
clips = []
berts = []
similairies = []

module_length = 10
num_beams = 1
num_return_sequences = 1

eval_device = "cuda:1"

for i, task_description in enumerate(tqdm(test_tasks)):
    print(task_description)
    task_rewards = []
    with torch.no_grad():
        input_s = ["### Human: "+ task_description +"\n### Assistant:"]
        input_ids = tokenizer.batch_encode_plus(
            input_s, padding="longest", return_tensors="pt"
        )["input_ids"].to(eval_device)
        output = model.generate(
            input_ids=input_ids,
            max_length=2048, 
            return_dict_in_generate=True, 
            output_scores=True, 
            output_hidden_states=True,
            repetition_penalty=1.25
        )
    generated_seq = tokenizer.decode(
        output["sequences"][0], skip_special_tokens=True, temperature=0, top_p=0.8, repetition_penalty=1.25
    )

    # print(generated_seq)

    # print(generated_seq.split("### Human: "))
    
    llama_solution = generated_seq.split("### Human: ")[1]
    print(llama_solution)
    llama_solution = llama_solution.split("\n")

    llama_steps = []
    for l,j in enumerate(llama_solution):
        if j[0:4] == "Step":
            llama_steps.append(llama_solution[l])
    print(llama_steps)
                
    module_list = match_module_seq(llama_steps, sentence_model)
    print("Module list: ", module_list)
    
    # vicuna_steps = generate_module_list_with_gpt(generated_seq).split(",")
    # module_list = match_module_seq(vicuna_steps, sentence_model)
    # print(module_list)

    # if len(module_list) >= 1 and module_seq_filter(module_list, test_task_idx[i]):
    if len(module_list) >= 1 and whole_module_seq_filter(module_list, test_task_idx[i]):
        seqCombination.construct_module_seq(module_list)

        for idx, batch in tqdm(enumerate(test_dataloaders[i])):
            inputs = list(batch['input'][0])
            # print("Inputs: ", inputs)
            try:
                predictions = seqCombination.run_module_seq(inputs)
            except:
                ave_task_reward = 0
                break

            if 0 <= test_task_idx[i] <= 14:
                outputs = list(batch['output'][0])
                dist = image_similarity(predictions, outputs, vit, vit_extractor)
                task_rewards.append(dist / 100)
            elif 15 <= test_task_idx[i] <= 104 or 107 <= test_task_idx[i]:
                outputs = list(batch['output'][0])
                f1 = np.mean(txt_eval(predictions, outputs, bertscore, device=eval_device))
                
                task_rewards.append(f1)
            else:
                score = clip_score(predictions, inputs)
                task_rewards.append(score.detach() / 100)
                
        ave_task_reward = np.mean(task_rewards)    
        seqCombination.close_module_seq()
            
    else:
        ave_task_reward = 0

    print(ave_task_reward)
        
    if 0 <= test_task_idx[i] <= 14:
        similairies.append(ave_task_reward)
    elif 15 <= test_task_idx[i] <= 104 or 107 <= test_task_idx[i]:
        berts.append(ave_task_reward)
    else:
        clips.append(ave_task_reward)

    rewards.append(ave_task_reward)     
    

print("Finished testing!")    

  0%|                                                    | 0/14 [00:00<?, ?it/s]

Given low-resolutioned blurry grayscale image, how to return the regular image step by step?
Given low-resolutioned blurry grayscale image, how to return the regular image step by step?
### Assistant: Sure! Here's a step-by-step guide on how to enhance and sharpen a low-resolutioned blurry grayscale image using various techniques. Please note that these are general steps and may require some adjustments based on the specific image you're working with.

Step 1: Noise Reduction
To start, we need to reduce the noise in the image. We can use a technique called "Gaussian Blur" to smooth out the image and remove any unwanted noise. This will help us improve the overall quality of the image before moving on to other steps.

Step 2: Sharpening
Once we have reduced the noise, we can move on to sharpening the image. There are several ways to do this, but one effective method is to use the "Unsharp Mask" filter. This filter will help bring out the details in the image and make it look more define

  7%|███                                        | 1/14 [02:09<27:59, 129.16s/it]

Module list Image Denoising, Colorization, Object Detection
0
Given blurry grayscale image, how to return the regular image step by step?
Given blurry grayscale image, how to return the regular image step by step?
### Assistant: Sure! Here's a step-by-step guide on how to enhance and sharpen a blurry grayscale image using Python:

Step 1: Load the Image
---------------

First, load the blurry grayscale image into your Python script. You can use libraries like OpenCV or Pillow for this purpose. For example:
```python
import cv2

blurry_image = cv2.imread('path/to/blurry/image', cv2.IMREAD_GRAYSCALE)
```
Step 2: Apply Bilateral Filtering
---------------------------

Bilateral filtering is an effective way to reduce noise in images while preserving edges. It works by weighting pixels based on their similarity in intensity and spatial distance. To apply bilateral filtering, you can use the following code:
```python
bilateral_filtered_image = cv2.bilateralFilter(blurry_image, 5, 90, 90)
```

 14%|██████▏                                    | 2/14 [06:14<39:28, 197.38s/it]

...

0.44406582504510883
Given English text, how to generate a image step by step?


 93%|███████████████████████████████████████   | 13/14 [31:04<02:04, 124.46s/it]

Given English text, how to generate a image step by step?
### Assistant: Sure! Here's an outline of the steps involved in generating an image from given English text using Deep Learning techniques:

1. Text Preprocessing: The input text is preprocessed to convert it into a numerical representation that can be fed into a neural network. This includes tokenization, stemming or lemmatizing, removing stop words and punctuation, and converting all texts to lowercase.
2. Tokenizer: A tokenizer is used to break down the text into individual tokens, such as words or phrases. These tokens are then fed into the next step.
3. Embeddings: Each token is embedded into a vector space using a learned embedding layer. This allows the model to understand the meaning of each word in relation to other words.
4. Encoder: An encoder module takes the embedded tokens and generates a continuous representation of the input text. This could be done using Recurrent Neural Network (RNN) like LSTM or GRU or Transfo


0it [00:00, ?it/s]
1it [00:08,  8.98s/it]
2it [00:15,  7.30s/it]
3it [00:21,  6.81s/it]
4it [00:29,  7.15s/it]
5it [00:40,  8.07s/it]
100%|██████████████████████████████████████████| 14/14 [32:57<00:00, 141.24s/it]

0.1968381442129612
Finished testing!


In [9]:
avg_clips = np.mean(clips)
avg_berts = np.mean(berts)
avg_similarities = np.mean(similairies)
avg_rewards = (avg_clips + avg_berts + avg_similarities) / 3
print([avg_clips, avg_berts, avg_similarities, avg_rewards])

[0.0, 0.2549198394417763, 0.2507233479817708, 0.1685477291411824]
